In [1]:
import tensorflow as tf

In [3]:
%pwd

'c:\\Users\\rahul\\Downloads\\sign_language_conversion\\research'

In [4]:
import os 
os.chdir("../")

In [5]:
%pwd

'c:\\Users\\rahul\\Downloads\\sign_language_conversion'

In [7]:
model = tf.keras.models.load_model("model/model.h5")

In [26]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input

test_generator = ImageDataGenerator(preprocessing_function=preprocess_input)

test_data = test_generator.flow_from_directory(
    'artifacts\\data_ingestion\\dataset\\Test',
    target_size=(224, 224),
    color_mode="rgb",
    batch_size=32,
    class_mode="categorical",
    shuffle=False
)

Found 112 images belonging to 28 classes.


In [27]:
model.evaluate(test_data,batch_size=32)

c:\Users\rahul\Downloads\sign_language_conversion\.venv\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


4/4 ━━━━━━━━━━━━━━━━━━━━ 28s 6s/step - accuracy: 0.9624 - loss: 0.1081


[0.12832129001617432, 0.9553571343421936]

In [15]:
import os

def get_classes_from_dataset(dataset_path):
    """
    Traverses the dataset directory and creates an array of class names.
    
    Args:
    - dataset_path (str): Path to the dataset directory.
    
    Returns:
    - class_names (list): List of class names.
    """
    # Check if the dataset path exists
    if not os.path.exists(dataset_path):
        raise ValueError(f"Dataset path '{dataset_path}' does not exist.")
    
    # List all subdirectories in the dataset path
    class_names = []
    for item in os.listdir(dataset_path):
        item_path = os.path.join(dataset_path, item)
        if os.path.isdir(item_path):
            class_names.append(item)
    
    return class_names

# Example usage
dataset_path = 'artifacts\data_ingestion\dataset\Train'
class_names = get_classes_from_dataset(dataset_path)
print("Classes in the dataset:", class_names)

Classes in the dataset: ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'Nothing', 'O', 'P', 'Q', 'R', 'S', 'Space', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z']


In [22]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input, decode_predictions
import numpy as np
import os

In [24]:

def predict_image_class(model, img_path, class_names):
    """
    Predicts the class of an image using the provided model.
    
    Args:
    - model: The pre-trained model.
    - img_path (str): Path to the image to be predicted.
    - class_names (list, optional): List of class names corresponding to the model's output. Only needed if not using ImageNet classes.
    
    Returns:
    - predicted_class (str): The predicted class name.
    """
    # Load and preprocess the image
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_input(img_array)
    
    # Predict the class
    predictions = model.predict(img_array)
    
    if class_names:
        # If custom class names are provided, use them
        predicted_class_index = np.argmax(predictions, axis=1)[0]
        predicted_class = class_names[predicted_class_index]
    else:
        # Use ImageNet class names
        decoded_predictions = decode_predictions(predictions, top=1)
        predicted_class = decoded_predictions[0][0][1]  # Get the class name
    
    return predicted_class

image_path = "artifacts\\data_ingestion\\dataset\\Train\\L\\7.jpg"

predict_image_class(model,image_path,class_names)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step


'L'

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Flatten, Dense, Dropout
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
import os

batch_size = 32 

train_generator = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    brightness_range=(0.1, 0.4),
    zoom_range=0.4,
    channel_shift_range=0.2,
    fill_mode='nearest',
    horizontal_flip=True,
    vertical_flip=True,
)

test_generator = ImageDataGenerator(preprocessing_function=preprocess_input)

train_data = train_generator.flow_from_directory(
    '/teamspace/studios/this_studio/dataset/Train',
    target_size=(224, 224),
    color_mode="rgb",
    batch_size=batch_size,
    class_mode="categorical"
)

test_data = test_generator.flow_from_directory(
    '/teamspace/studios/this_studio/dataset/Test',
    target_size=(224, 224),
    color_mode="rgb",
    batch_size=batch_size,
    class_mode="categorical",
    shuffle=False
)

input_shape = (224, 224, 3)
inputs = Input(shape=input_shape)

pretrained_model = VGG16(include_top=False, input_shape=input_shape, weights='imagenet', input_tensor=inputs)

for layer in pretrained_model.layers:
    layer.trainable = False

block_output = pretrained_model.get_layer('block5_pool').output

x = Flatten()(block_output)

x = Dense(512, activation='relu', kernel_initializer='he_uniform')(x)
x = Dropout(0.4)(x)
x = Dense(216, activation='relu', kernel_initializer='he_uniform')(x)
output = Dense(28, activation='softmax', kernel_initializer='he_uniform')(x)

model = Model(inputs=inputs, outputs=output)

model.load_weights('checkpoints/weights.008-0.9911.weights.h5')

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
